# Install and Load Packages

In [1]:
!pip install transformers
!pip install datasets
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 27.5 MB/s 
     |████████████████████████████████| 182 kB 68.2 MB/s 
     |████████████████████████████████| 7.6 MB 57.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 30.1 MB/s 
     |████████████████████████████████| 212 kB 81.1 MB/s 
     |████████████████████████████████| 132 kB 79.0 MB/s 
     |████████████████████████████████| 127 kB 73.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 86 kB/s 


In [2]:
import torch
import torch.nn as nn
import tqdm
import pandas as pd
import faiss
import faiss.contrib.torch_utils

from transformers import DPRContextEncoder, DPRContextEncoderTokenizerFast
from datasets import Dataset
from google.colab import auth, drive
from google.cloud import bigquery

In [3]:
auth.authenticate_user()
print('Authenticated')

drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/nlp/data/'

Authenticated
Mounted at /content/drive


# Load Data

In [4]:
project_id = 'calcium-vial-368801'
client = bigquery.Client(project=project_id)

In [5]:
dt_train_queries = client.query('''
SELECT DISTINCT query_id, doc_id, title, text, questions
FROM `calcium-vial-368801.staging.nq_train_documents_2_qg_25_beam`
''').to_dataframe()

In [6]:
len(dt_train_queries)

26634

In [7]:
doc_titles = dt_train_queries[['doc_id', 'title']]

In [8]:
dt_train_queries['passage_append'] = dt_train_queries['title'] + ' [SEP] ' + dt_train_queries['text']

In [9]:
queries_text = client.query('''
SELECT DISTINCT query_id, text AS query
FROM `calcium-vial-368801.beir_nq_train.train_query_lookup`
''').to_dataframe()

In [10]:
dt_train_clean = pd.merge(
    dt_train_queries,
    queries_text,
    on='query_id',
    how='left'
)[["query_id", "query", "passage_append"]].drop_duplicates()

# Data Loader

In [11]:
ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/492 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [12]:
class MyDataset(Dataset):
    def __init__(self, dataframe, p_tokenizer):
        self.dataframe = dataframe
        self.p_tokenizer = p_tokenizer

        self.p_embed = p_tokenizer(
            self.dataframe['passage_append'].tolist(),
            return_tensors='pt',
            truncation=True,
            max_length=512,
            padding='max_length'
        )

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        return self.p_embed[index]


def collate_fn(batch):
    batchsize = len(batch)

    ctx_tensor = torch.LongTensor(
        [[sample.ids, sample.attention_mask, sample.type_ids] for sample in batch]
        )

    return ctx_tensor

BATCH_SIZE = 20

dataloader_train = torch.utils.data.DataLoader(
    MyDataset(dt_train_clean, ctx_tokenizer),
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn
    )

In [13]:
import gc

del ctx_tokenizer

gc.collect()
torch.cuda.empty_cache()

# Model

In [14]:
# https://discuss.huggingface.co/t/finetuning-dpr-on-custom-dataset/4170
ctx_model = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base').to("cuda")

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [15]:
gc.collect()
torch.cuda.empty_cache()

In [16]:
res = faiss.StandardGpuResources()
index = faiss.IndexFlatL2(768)
gpu_index = faiss.index_cpu_to_gpu(res, 0, index)

In [17]:
class PassageEncoder(nn.Module):
    def __init__(self, p_encoder, index):
        super().__init__()
        self.p_encoder = p_encoder
        self.index = index

    def forward(self, passage):
        self.index.add(self.p_encoder(passage[:, 0, :], passage[:, 1, :], passage[:, 2, :]).pooler_output.contiguous())


In [18]:
pEncoder = PassageEncoder(ctx_model, gpu_index)

# 5327 total batches
for i in tqdm.notebook.tqdm(dataloader_train, total=len(dataloader_train)):
    pEncoder(i.to("cuda"))
    torch.cuda.empty_cache()

  0%|          | 0/1332 [00:00<?, ?it/s]

In [19]:
faiss.write_index(faiss.index_gpu_to_cpu(gpu_index), data_path + 'nq_train_passage_encodings/nq_train_2_no_qg_beam_passage_index')
print(gpu_index.ntotal)

26634


In [20]:
import json

idx2docid = dt_train_queries['doc_id'].to_dict()

with open(data_path + 'nq_train_passage_encodings/nq_train_2_no_qg_beam_passage_idx2docid.json', 'w') as f:
   f.write(json.dumps(idx2docid))